# Decision trees tryouts on SPR data, inspired by Kaggle Forum "When less is more"

Load training and validation data as 
    month : [ Features | Targets| Difference | Last Choice Targets  ]
    
    - Features : 
    - Target labels : TARGET_LABELS(month), 'targets_str', 'targets_features'
    - Difference with prev month
    - Last Choice Targets : LC_TARGET_LABELS(month-1), 'lc_targets_str', 'lc_targets_features'


In [ ]:
import os
import numpy as np
import pandas as pd

import logging
logging.getLogger().handlers = []
logging.basicConfig(level=logging.INFO)

import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
from dataset import load_trainval, LC_TARGET_LABELS, TARGET_LABELS_FRQ, TARGET_LABELS_DIFF, decimal_to_dummies, targets_str_to_indices, targets_dec_to_indices
from common import to_yearmonth, TARGET_LABELS 
from common import target_str_to_labels, TARGET_LABELS2

In [ ]:
features = [
    u'ind_empleado', u'pais_residencia',
    u'sexo', u'age', u'ind_nuevo', u'antiguedad', u'indrel',
    u'ult_fec_cli_1t', u'indrel_1mes', u'tiprel_1mes', u'indresi',
    u'indext', u'conyuemp', u'canal_entrada', u'indfall', u'nomprov',
    u'ind_actividad_cliente', u'renta', u'segmento'    
]

In [ ]:
# train_yearmonths_list = [201504, 201505, 201604]
train_yearmonths_list = [201505, 201605]
# train_yearmonths_list = [201505]
val_yearmonth = [201605]
train_nb_clients = 150000
# train_nb_clients = 1500
train_df, val_df = load_trainval(train_yearmonths_list, val_yearmonth, train_nb_clients, val_nb_clients=1500)

Display loaded data

In [ ]:
train_df[['fecha_dato', 'ncodpers'] + TARGET_LABELS_DIFF.tolist()].head(10)

Useful structures

In [ ]:
def get_common_clients(df1, mask1, mask2, df2=None):
    active_clients1 = df1[mask1]['ncodpers'].unique()
    if df2 is not None:
        active_clients2 = df2[mask2]['ncodpers'].unique()
    else:
        active_clients2 = df1[mask2]['ncodpers'].unique()
    active_clients = list(set(active_clients1) & set(active_clients2)) 
    
    if df2 is not None:
        return df1['ncodpers'].isin(active_clients), df2['ncodpers'].isin(active_clients)
    return df1['ncodpers'].isin(active_clients)

In [ ]:
months_ym_map = {}
months = list(set(train_df['fecha_dato'].unique()) | set(val_df['fecha_dato'].unique()))
for m in months:
    months_ym_map[to_yearmonth(m)] = m

        
train_months = train_df['fecha_dato'].unique()
val_months = val_df['fecha_dato'].unique()
    

In [ ]:
from common import get_added_products, remove_last_choice, apk, map7_score
from visualization import visualize_train_test, visualize_folds, compare_two_datasets, compare_folds, compare_folds2

Create profiles and create models for profiles

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Merge
from keras.utils import np_utils

### Train a model

In [ ]:
target_features = ['targets_diff', 'targets_logdiff', 'targets_logcount2_diff', 'targets_logcount2', 'targets_logcount1', 'targets_logDec']

In [ ]:
#active_clients_mask = ~train_df['targets_diff'].isin([0, -99999])
#print active_clients_mask.shape, active_clients_mask.sum()

In [ ]:
def get_XY(current_month, df1, next_year_month, df2, months_ym_map):
    month_mask = df1['fecha_dato'] == months_ym_map[current_month]
    next_year_month_mask = df2['fecha_dato'] == months_ym_map[next_year_month]
    
    # get common clients from df1 at this month and df2 at next year month
    common_clients_mask1, common_clients_mask2 = get_common_clients(df1, month_mask, next_year_month_mask, df2)
    
    X = df1[common_clients_mask1 & month_mask][['ncodpers', 'fecha_dato'] + target_features + features + TARGET_LABELS_FRQ.tolist()]            

    if TARGET_LABELS[0] in df2.columns and TARGET_LABELS_DIFF[0] in df2.columns:
        Y = df2[common_clients_mask2 & next_year_month_mask][['ncodpers', 'fecha_dato', 'targets_str'] + TARGET_LABELS + TARGET_LABELS_DIFF.tolist()]    
        assert (X['ncodpers'].values == Y['ncodpers'].values).all(), "There is a problem in alignment"
        Y.index = X.index
    else:
        Y = None
    
    if LC_TARGET_LABELS[0] in df2.columns:
        clients_last_choice = df2[common_clients_mask2 & next_year_month_mask][['ncodpers', 'fecha_dato'] + LC_TARGET_LABELS.tolist()]
    else:
        clients_last_choice = None
        
    return X, Y, clients_last_choice


In [ ]:
current_month = 201505
next_year_month = current_month + 100

df1 = train_df if months_ym_map[current_month] in train_months else val_df
#df1 = train_df
df2 = train_df if months_ym_map[next_year_month] in train_months else val_df
#df2 = train_df

X, Y, clients_last_choice = get_XY(current_month, df1, next_year_month, df2, months_ym_map)

In [ ]:
print X.shape
X.head(10)

In [ ]:
print Y.shape
Y.head(10)

In [ ]:
print clients_last_choice.shape
clients_last_choice.head(10)

### Run KFold Cross-validation 

In [2]:
help(DecisionTreeClassifier)

Help on class DecisionTreeClassifier in module sklearn.tree.tree:

class DecisionTreeClassifier(BaseDecisionTree, sklearn.base.ClassifierMixin)
 |  A decision tree classifier.
 |  
 |  Read more in the :ref:`User Guide <tree>`.
 |  
 |  Parameters
 |  ----------
 |  criterion : string, optional (default="gini")
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |  
 |  splitter : string, optional (default="best")
 |      The strategy used to choose the split at each node. Supported
 |      strategies are "best" to choose the best split and "random" to choose
 |      the best random split.
 |  
 |  max_features : int, float, string or None, optional (default=None)
 |      The number of features to consider when looking for the best split:
 |  
 |          - If int, then consider `max_features` features at each split.
 |          - If float, then `max_features` is a percentage and
 |

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


def create_model(profiles, ll):
    
    model = DecisionTreeClassifier()
    return model

In [ ]:
common_group1 = [2, 18, 23] #range(24)
common_group2 = [3, 4, 7, 8, 12] #range(24)
others = list(set(range(24)) - set(common_group1 + common_group2))
for i, a in enumerate(TARGET_LABELS2[common_group1]):
    print i, a

In [ ]:
# target_groups = [common_group1, common_group2, others]
target_groups = [common_group1]
print sum([len(t) for t in target_groups])

In [ ]:
not_normalized_profiles = []

profiles = {
    
## 
    0: ['pais_residencia', 'sexo', 'age', 'ind_nuevo', 'segmento', 'ind_empleado', 'ind_actividad_cliente', 'indresi'],
#   100: ['pais_residencia', 'sexo', 'age', 'ind_nuevo', 'segmento', 'ind_empleado', 'ind_actividad_cliente', 'indresi'],

## 
    1: target_features,
#     101: target_features,
    
    2: ['pais_residencia', 'sexo', 'age', 'segmento', 'renta'],
    3: ['pais_residencia', 'sexo', 'age', 'renta', 'targets_logdiff', 'targets_logcount2_diff','targets_logcount2','targets_logcount1'],
    11: ['nomprov', 'ind_nuevo', 'renta', 'ind_actividad_cliente', 'canal_entrada'],

## 
     12: TARGET_LABELS_FRQ,
#     112: TARGET_LABELS_FRQ,        
    
##    12: TARGET_LABELS_FRQ[common_group1 + common_group2],
##    112: TARGET_LABELS_FRQ[common_group1 + common_group2],        
    
     13: list(TARGET_LABELS_FRQ[common_group1 + common_group2]) + target_features,
#     113: list(TARGET_LABELS_FRQ[common_group1 + common_group2]) + target_features,

#     14: ['pais_residencia', 'sexo', 'age', 'segmento', 'renta'] + list(TARGET_LABELS_FRQ[common_group])
#     114: ['pais_residencia', 'sexo', 'age', 'segmento', 'renta'] + list(TARGET_LABELS_FRQ[common_group])
}

In [ ]:
from sklearn.preprocessing import StandardScaler

def prepare_to_fit(X_train, Y_train, profiles, mask):
    X_train_ = X_train[mask]
    y_train = Y_train[mask].values
    x_train = []
    for key in profiles:
        x_train_ = X_train_[profiles[key]].values
        if key not in not_normalized_profiles:
            x_train_ = StandardScaler().fit_transform(x_train_)
        x_train.append(x_train_)
    return x_train, y_train


def prepare_to_test(X_val, profiles, Y_val=None):
    
    y_val = Y_val.values if Y_val is not None else None
    x_val = []
    for key in profiles:
        x_val_ = X_val[profiles[key]].values   
        if key not in not_normalized_profiles:
            x_val_ = StandardScaler().fit_transform(x_val_)  
        x_val.append(x_val_)    
    
    return x_val, y_val
    

In [ ]:
n_highest = 7
    
def pred_to_targets_indices(y_probas, **kwargs):
    
    threshold = 0.5 if 'threshold' not in kwargs else kwargs['threshold']
    n_highest = 7 if 'n_highest' not in kwargs else kwargs['n_highest']
    
    y_probas[y_probas < threshold] = 0.0
    predicted_added_products = np.argsort(y_probas, axis=1)
    predicted_added_products = predicted_added_products[:,::-1][:,:n_highest]
    out = []
    index_map = lambda x: x if 'index_map' not in kwargs else kwargs['index_map'][x]
    for i, t in enumerate(predicted_added_products):
        out.append([index_map(j) for j in t if y_probas[i, j] > 0.0])
    return np.array(out)


def train_model0(X_train, Y_train, profiles, 
                prepare_to_fit_func=prepare_to_fit, 
                create_model_func=create_model,
                **kwargs):
    
    # Select only active client for the training part
    mask = ~X_train['targets_diff'].isin([0, -99999])

    x_train, y_train = prepare_to_fit_func(X_train, Y_train, profiles, mask)
    logging.info("- Train data shapes : {}, {}".format(
            [i.shape for i in x_train] if isinstance(x_train, list) else x_train.shape, 
            y_train.shape)
    )                
    logging.info("- Create the model")
    estimator = create_model_func(profiles, y_train.shape[1])
    
    nb_epoch = 100 if 'nb_epoch' not in kwargs else kwargs['nb_epoch']
    batch_size = min(7500, x_train[0].shape[0]) if 'batch_size' not in kwargs else kwargs['batch_size']

    logging.info("- Fit the model : (%i, %i)" % (nb_epoch, batch_size))
    hist = estimator.fit(x_train, y_train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=0)        
    for key in hist.history:            
        logging.info("-- {} : min={:5f}, max={:5f}, last={}".format(key, np.min(hist.history[key]), np.max(hist.history[key]), hist.history[key][-4:]))
    return estimator


def train_model(X_train, Y_train, profiles, 
                prepare_to_fit_func=prepare_to_fit, 
                create_model_func=create_model,
                **kwargs):
    
    estimators = []
    for group in target_groups:
        logging.info("--- process group : {}".format(group))
        target_labels = np.array(TARGET_LABELS)[group]
        estimator = train_model0(X_train, Y_train[target_labels], profiles,
                                 create_model_func=create_model,
                                 prepare_to_fit_func=prepare_to_fit, 
                                 **kwargs)  
        estimators.append(estimator)            
    return estimators


def predict_with_model0(estimator, X_val, profiles, Y_val=None, 
                        prepare_to_test_func=prepare_to_test, 
                        transform_pred_func=pred_to_targets_indices, 
                       **kwargs):
    """
    kwargs : 
        threshold: e.g. 0.55
        return_probas: True/False
        group (list of indices)
    """
    logging.info("- Predict using trained model")
    
    Y_val = Y_val[TARGET_LABELS] if Y_val is not None else None
    x_val, y_val = prepare_to_test_func(X_val, profiles, Y_val)
    
    logging.info("- Test data shapes : {}, {}".format(
            [i.shape for i in x_val] if isinstance(x_val, list) else x_val.shape, 
            y_val.shape if y_val is not None else '')
    )                
    
    y_probas = estimator.predict(x_val, verbose=0)    
    y_pred = transform_pred_func(y_probas, **kwargs)
    
    return_probas = kwargs['return_probas'] if 'return_probas' in kwargs else False    
    
    if y_val is not None:
        y_val = targets_str_to_indices(y_val, **kwargs)
        
    if return_probas:
        return y_pred, y_val, y_probas
    return y_pred, y_val
    

def predict_with_model(estimators, X_val, profiles, Y_val=None, 
                        prepare_to_test_func=prepare_to_test, 
                        transform_pred_func=pred_to_targets_indices, 
                       **kwargs):
    """
    kwargs : 
        threshold: e.g. 0.55
        return_probas: True/False
        group (list of indices)
    """
    logging.info("- Predict using trained model")
    
    Y_val = Y_val[TARGET_LABELS] if Y_val is not None else None
    x_val, y_val = prepare_to_test_func(X_val, profiles, Y_val)
    logging.info("- Test data shapes : {}, {}".format(
            [i.shape for i in x_val] if isinstance(x_val, list) else x_val.shape, 
            y_val.shape if y_val is not None else ''))          
    if y_val is not None:
        y_val = targets_str_to_indices(y_val)
    
    _y_probas = []   
    clc_val = None if 'clc_val' not in kwargs else kwargs['clc_val']
    
    for i, group in enumerate(target_groups):
        logging.info("--- process group : {}".format(group))
        target_labels = np.array(TARGET_LABELS)[group]
            
        y_probas = estimators[i].predict(x_val, verbose=0)            
        
        if clc_val is not None and y_val is not None:
            y_subpred = transform_pred_func(y_probas, index_map=group)
            subscore = map7_score(y_val, y_subpred, clc_val)
            logging.info("--- group subscore: {}".format(subscore))
        
        _y_probas.append(y_probas)
    
    y_probas = merge_predictions(_y_probas, target_groups)
    y_pred = transform_pred_func(y_probas, **kwargs)
    return_probas = kwargs['return_probas'] if 'return_probas' in kwargs else False    
    
    if return_probas:
        return y_pred, y_val, y_probas
    return y_pred, y_val
    

def merge_predictions(y_array, groups):
    """
    y_array is a list of ndarrays : [y1, y2, ...]
        yi can be of shape (nb_samples, len(gi)) or (nb_samples,)
    groups is a list of indices : [g1, g2, ..., gN], len(y_array) == len(groups)
    return a properly ordered ndarray of shape (nb_samples, len(TARGET_LABELS)) or (nb_samples,) 
    """
    #ll = np.sum([len(g) for g in groups])
    ll = len(TARGET_LABELS)
    nb_samples = y_array[0].shape[0]
    two_d_case = len(y_array[0].shape) == 2    
    if two_d_case:
        out = np.zeros((nb_samples, ll)) 
    else:
        out = np.empty((nb_samples, ), dtype=np.object_)
        out.fill([])
    for g, y in zip(groups, y_array):
        if two_d_case:
            out[:,g] = y[:]
        else:
            out += y
    return out

In [ ]:
# CROSS VALIDATION
from sklearn.model_selection import KFold
def cross_val_score2(data, 
                     profiles,
                     nb_folds=5, 
                     train_model_func=train_model,
                     predict_with_model_func=predict_with_model):
    
    logging.info("- Cross validation : ")
    x_df, y_df, clients_last_choice = data
    kf = KFold(n_splits=nb_folds)
    scores = []
    
    count = 0
    for train_index, test_index in kf.split(range(x_df.shape[0])):
        count += 1
        logging.info("\n\n\t\t-- Fold : %i / %i\n" % (count, nb_folds))
        
        X_train, X_val = x_df.loc[x_df.index[train_index], :], x_df.loc[x_df.index[test_index], :]
        Y_train, Y_val = y_df.loc[y_df.index[train_index], :], y_df.loc[y_df.index[test_index], :]
        clc_val = clients_last_choice[test_index, :]

        estimators = train_model_func(X_train, Y_train, profiles, nb_epoch=300)    
        y_pred, y_val = predict_with_model_func(estimators, X_val, profiles, Y_val, 
                                                clc_val=clc_val, 
                                                threshold=0.5, 
                                                n_highest=7)        
        
        logging.info("- Compute map7 score")
        scores.append(map7_score(y_val, y_pred, clc_val))   
        
    return np.array(scores)

In [ ]:
nb_folds = 3
results = cross_val_score2((X, Y, clients_last_choice[LC_TARGET_LABELS].values), 
                            profiles=profiles,
                            nb_folds=nb_folds)
print "Profiles : ", profiles.keys()
print "Cross-Validation \n %i | %f | %f | %f | %.5f " % (nb_folds, results.min(), results.mean(), results.max(), results.std())



### 201505 -> 201605 

#### Single profiles:

Profiles   [0, 100] 
Cross-Validation (normalized)
 3 | 0.011683 | 0.012817 | 0.014958 | 0.00152 

Cross-Validation (not normalized)
 3 | 0.009244 | 0.010407 | 0.011922 | 0.00112 
 
 
Profiles :  [1, 101]
Cross-Validation (normalized)
 3 | 0.006793 | 0.009161 | 0.012219 | 0.00227 

Cross-Validation (not normalized)
 3 | 0.004787 | 0.009852 | 0.014922 | 0.00414


Profiles :  [112, 12]
Cross-Validation (normalized)
 3 | 0.008856 | 0.012124 | 0.016443 | 0.00318 

Cross-Validation (not normalized)
 3 | 0.007298 | 0.010140 | 0.014101 | 0.00289 

Compute cross-validation across several months

In [ ]:
nb_folds = 3
yms = [201504, 201505]
#yms = [201505]

for ym in yms:
    logging.info("\n-------------------------")
    logging.info("- Process month : %s" % ym)
    logging.info("-------------------------\n")
    
    ym1 = ym + 100    
    df1 = train_df if months_ym_map[ym] in train_months else val_df
    df2 = train_df if months_ym_map[ym1] in train_months else val_df
    X, Y, clients_last_choice = get_XY(ym, df1, ym1, df2) 
    results = cross_val_score2((X, Y, clients_last_choice[LC_TARGET_LABELS].values), 
                                profiles=profiles,
                                nb_folds=nb_folds)
    print "Cross-Validation \n %i | %f | %f | %f | %.5f " % (nb_folds, results.min(), results.mean(), results.max(), results.std())

## Train model for predictions

In [ ]:
current_month = 201505
next_year_month = current_month + 100

df1 = train_df
#df1 = val_df
df2 = train_df #if months_ym_map[next_year_month] in train_months else val_df
#df2 = val_df

X, Y, clients_last_choice = get_XY(current_month, df1, next_year_month, df2, months_ym_map)

In [ ]:
estimators = train_model(X, Y, profiles, nb_epoch=300)

Check score on the data 2016/05

In [ ]:
y_pred, y_val, y_probas = predict_with_model(estimators, X, profiles, Y, threshold=0.5, return_probas=True)

logging.info("- Compute map7 score")
print map7_score(y_val, y_pred, clients_last_choice[LC_TARGET_LABELS].values)
logging.info("- Compute max map7 score")
print map7_score(y_val, y_val, clients_last_choice[LC_TARGET_LABELS].values)

In [ ]:
from common import targets_to_labels, targets_indices_to_labels, remove_last_choice

In [ ]:
clients_last_choice[LC_TARGET_LABELS].values

limit = 100
count = 0
for last_choice, targets, products in zip(clients_last_choice[LC_TARGET_LABELS].values, y_val, y_pred):
    added_products = remove_last_choice(targets, last_choice)
    predictions = remove_last_choice(products, last_choice)
    
    if len(added_products) == 0:
        continue
        
    #print added_products, common_group1, len(set(added_products) & set(common_group1))
    if len(set(added_products) & set(target_groups[0])) < 1:
        continue
        
    print "--- Count = ", count
    print targets_indices_to_labels(added_products, TARGET_LABELS2)
    print targets_indices_to_labels(predictions, TARGET_LABELS2), targets_indices_to_labels(products, TARGET_LABELS2)
    
    count += 1
    if count == limit:
        break

In [ ]:
print y_probas[:10, target_groups[0]]
print Y[np.array(TARGET_LABELS)[target_groups[0]]].head(10)

## Prediction for 2016/06

In [ ]:
from dataset import load_train_test

In [ ]:
full_train_df, test_df = load_train_test([201506])

In [ ]:
full_train_df.head()

In [ ]:
test_df.head()

In [ ]:
months_ym_map = {}
months = list(set(full_train_df['fecha_dato'].unique()) | set(test_df['fecha_dato'].unique()))
for m in months:
    months_ym_map[to_yearmonth(m)] = m
    
full_train_months = full_train_df['fecha_dato'].unique()
test_months = test_df['fecha_dato'].unique()

In [ ]:
def get_XY(current_month, df1, next_year_month, df2, months_ym_map):
    month_mask = dfx1['fecha_dato'] == months_ym_map[current_month]
    next_year_month_mask = df2['fecha_dato'] == months_ym_map[next_year_month]
    
    # get common clients from df1 at this month and df2 at next year month
    common_clients_mask1, common_clients_mask2 = get_common_clients(df1, month_mask, next_year_month_mask, df2)
    
    X = df1[common_clients_mask1 & month_mask][['ncodpers', 'fecha_dato'] + target_features + features + TARGET_LABELS_FRQ.tolist()]            

    if TARGET_LABELS[0] in df2.columns:
        Y = df2[common_clients_mask2 & next_year_month_mask][['ncodpers', 'fecha_dato', 'targets_str'] + TARGET_LABELS]    
        assert (X['ncodpers'].values == Y['ncodpers'].values).all(), "There is a problem in alignment"
        Y.index = X.index
    else:
        Y = None
    
    if LC_TARGET_LABELS[0] in df2.columns:
        clients_last_choice = df2[common_clients_mask2 & next_year_month_mask][['ncodpers', 'fecha_dato'] + LC_TARGET_LABELS.tolist()]
    else:
        clients_last_choice = None
        
    return X, Y, clients_last_choice

In [ ]:
current_month = 201506
next_year_month = current_month + 100

df1 = full_train_df
df2 = test_df
X, _, clients_last_choice = get_XY(current_month, df1, next_year_month, df2, months_ym_map)

In [ ]:
print X.shape, test_df.shape

In [ ]:
X.head(10)

In [ ]:
clients_last_choice.head(10)

In [ ]:
def get_submission(predicted_added_products, clients, clc, target_labels):
    added_products_col = []
    count = 0 
    for products, last_choice in zip(predicted_added_products, clc):
        predictions = remove_last_choice(products, last_choice)
        added_products_col.append(' '.join([target_labels[i] for i in predictions]))
        count+=1
        if count % 100000 == 0:
            logging.info("Elapsed : %i", count)
            
    out = pd.DataFrame(data={'ncodpers': clients, 'added_products': added_products_col}, columns=['ncodpers', 'added_products'])
    return out

In [ ]:
y_pred = predict_with_model(estimators, X, profiles, threshold=0.5)

logging.info("- Get submission dataframe:")
clients = X['ncodpers'].values
submission = get_submission(y_pred, clients, clients_last_choice[TARGET_LABELS].values, TARGET_LABELS)

In [ ]:
submission_clients = set(submission['ncodpers'].unique())
test_clients = set(test_df['ncodpers'].unique())
if submission_clients != test_clients:
    missing_clients = list(test_clients - submission_clients)
    missing_added_products = np.zeros((len(missing_clients)))
    submission = pd.concat([submission, 
                            pd.DataFrame(data={
                                'ncodpers': missing_clients, 
                                'added_products': missing_added_products
                            }, columns=['ncodpers', 'added_products'])])

Get submission DataFrame and write csv file

In [ ]:
print submission.shape
submission.head()

In [ ]:
from datetime import datetime
import csv

logging.info('- Generate submission')
submission_file = '../results/submission_' + \
                  str(datetime.now().strftime("%Y-%m-%d-%H-%M")) + \
                  '.csv'

submission.to_csv(submission_file, index=False, index_label=False)

In [ ]:
with open('../results/submission_2016-11-17-16-37.csv', 'r') as r:
    print r.readline()
    print r.readline()
    print r.readline()
    print r.readline()
    print r.readline()
    